<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 2

### Iowa Liquor Sales 

---

In this project, you will be using the pandas data transformation functionality (melt, pivot, groupby, apply) that was not a factor in the first half of Project 2. 

---

#### Context of the data

The state of Iowa provides many data sets on their website, including [this dataset](https://www.dropbox.com/sh/pf5n5sgfgiri3i8/AACkaMeL_i_WgZ00rpxOOcysa?dl=0) which contains transactions for all stores that have a class E liquor license. You can choose one of the following two scenarios.

The data can also be found [directly on their website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), which allows you to explore it graphically and download it (though it doesn't work very well).

NOTE: Some of you may have computer issues with the full dataset. In this case, feel free to use [this 10% dataset version of Iowa liquor sales](https://drive.google.com/file/d/0Bx2SHQGVqWaseDB4QU9ZSVFDY2M/view?usp=sharing). You may want to use it anyway to test and prototype your code since it will be faster, before running it on the full dataset.

---

#### Package imports

In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import datetime as dt

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the data from file

---

In [2]:
# check the iowa file location for your computer 
iowa_file = '../../../DSI-SF-3/datasets/iowa_liquor/Iowa_Liquor_sales_sample_10pct.csv'

iowa = pd.read_csv(iowa_file)
print "Dataframe is of size: " + str(iowa.values.nbytes / 10**6) + "MB"

Dataframe is of size: 39MB


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Do an initial overview of the data

---

At the very least describe the columns/variables and the datatypes. 

In [3]:
iowa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Clean the dataset

---

### 3.1 Identify columns that you will need to convert and clean. Where and how is the data corrupted?

Don't worry about converting the date column to a pandas/numpy "datetime" datatype, unless you want to (not required for these problems and is a challenging thing to work with in its own right.)

In [4]:
iowa = iowa.rename(columns = {'Volume Sold (Gallons)' : 'Volume_Sold_Gallons'})
iowa = iowa.rename(columns = {'Volume Sold (Liters)' : 'Volume_Sold_Liters'})
iowa = iowa.rename(columns = {'Bottle Volume (ml)' : 'Bottle_Volume_ml'})
iowa = iowa.rename(columns = {'Sale (Dollars)' : 'Sale_Dollar'})
iowa.columns

iowa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                   270955 non-null object
Store Number           270955 non-null int64
City                   270955 non-null object
Zip Code               270955 non-null object
County Number          269878 non-null float64
County                 269878 non-null object
Category               270887 non-null float64
Category Name          270323 non-null object
Vendor Number          270955 non-null int64
Item Number            270955 non-null int64
Item Description       270955 non-null object
Bottle_Volume_ml       270955 non-null int64
State Bottle Cost      270955 non-null object
State Bottle Retail    270955 non-null object
Bottles Sold           270955 non-null int64
Sale_Dollar            270955 non-null object
Volume_Sold_Liters     270955 non-null float64
Volume_Sold_Gallons    270955 non-null float64
dtypes: float64(4), int64(5), object(9)
memory usage: 37.2+ MB

### 3.2 Clean the columns

In [5]:
iowa['State Bottle Retail'] = iowa['State Bottle Retail'].map(lambda x: x.lstrip('$').rstrip('aAbBcC')).astype('float')
iowa['State Bottle Cost'] = iowa['State Bottle Cost'].map(lambda x: x.lstrip('$').rstrip('aAbBcC')).astype('float')
iowa['Sale_Dollar'] = iowa['Sale_Dollar'].map(lambda x: x.lstrip('$').rstrip('aAbBcC')).astype('float')
iowa['Store Number'] = iowa['Store Number'].astype('str')
iowa['County Number'] = iowa['County Number'].astype('str')
iowa['Category'] = iowa['Category'].astype('str')
iowa['Item Number'] = iowa['Item Number'].astype('str')
iowa['Vendor Number'] = iowa['Vendor Number'].astype('str')
iowa['Date'] = pd.to_datetime(iowa['Date'])

iowa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                   270955 non-null datetime64[ns]
Store Number           270955 non-null object
City                   270955 non-null object
Zip Code               270955 non-null object
County Number          270955 non-null object
County                 269878 non-null object
Category               270955 non-null object
Category Name          270323 non-null object
Vendor Number          270955 non-null object
Item Number            270955 non-null object
Item Description       270955 non-null object
Bottle_Volume_ml       270955 non-null int64
State Bottle Cost      270955 non-null float64
State Bottle Retail    270955 non-null float64
Bottles Sold           270955 non-null int64
Sale_Dollar            270955 non-null float64
Volume_Sold_Liters     270955 non-null float64
Volume_Sold_Gallons    270955 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(2), o

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3.3 Perform more extensive cleaning of the dataset

Cleaning of data can mean a lot more than just fixing strings and numbers in columns. There are often logical errors with data, useless or nonsensical categories, redundancy of information, outliers, and many more problems.

This dataset has problems beyond just fixing the types of columns. Though resolving them may not be required for EDA and analysis, if you want experience with "deeper" cleaning of data this is a great dataset to start practicing with.

Keep in mind that some types of "data cleaning" is subjective: it's not always a cut-and-dry conversion of type or removal of null values. Subjectivity when dealing with data is just a fact of life for a data scientist. This isn't a kind of programming where things are just right or wrong.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Filter/adjust the store data

---

Some stores may have opened or closed in 2015. These stores will have incorrect yearly summary statistics since they were not open the full year. We need to filter them out or find another way to deal with the inconsistent numbers of months across stores.

It is up to you how you want to deal with this problem.

1. Investigate problematic stores.
2. Decide on an approach to deal with stores not open for the full 2015 year.
3. Implement your plan.
4. Briefly report on what you did and why.
iowa['sale_year'] = iowa.Date.dt.year

In [6]:
iowa['sale_year'] = iowa.Date.dt.year
iowa_summary = pd.pivot_table(iowa, index=['sale_year', 'Store Number'], values=['Sale_Dollar'],aggfunc=[np.sum])
iowa_summary.ix[2015].head()

,sum
,Sale_Dollar
Store Number,
2106,146326.22
2113,9310.22
2130,111871.43
2152,7721.08
2178,24324.18


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Proposals for new tax rates (Scenario 1)

---

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and has assigned you to the project.

### 5.1 Calculate yearly liquor sales for each store in 2015.

In [7]:
iowa_sales = pd.pivot_table(iowa, index=['sale_year','Store Number'], values=['Sale_Dollar'],aggfunc=[np.sum])
iowa_sales.ix[2015].head()

,sum
,Sale_Dollar
Store Number,
2106,146326.22
2113,9310.22
2130,111871.43
2152,7721.08
2178,24324.18


### 5.2 Calculate the profit each store is making in 2015.


In [8]:
iowa['Profit'] = ((iowa['State Bottle Retail'] - iowa['State Bottle Cost'])) * iowa['Bottles Sold']
iowa_profit = pd.pivot_table(iowa, index=['sale_year','Store Number'], values=['Profit'],aggfunc=[np.sum])
iowa_profit.ix[2015].head()
iowa.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle_Volume_ml,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale_Dollar,Volume_Sold_Liters,Volume_Sold_Gallons,sale_year,Profit
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0,2.38,2015,27.00
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.40,2016,13.76
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,6.34,2016,151.20
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.5,2.77,2016,28.50
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.0,5.55,2015,43.20


### 5.3 Investigate which Iowa counties are making the most profit on liquor per gallon in 2015.

In [9]:
iowa['Gallon_profit'] = iowa.Profit / iowa.Volume_Sold_Gallons
iowa.Gallon_profit.describe()
county_gallon_profit = iowa.groupby('County').sum()[['Gallon_profit']].round()
county_gallon_profit.sort('Gallon_profit', ascending=False).head()

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Gallon_profit
County,
Polk,1217319.0
Linn,553419.0
Scott,402966.0
Black Hawk,355928.0
Johnson,322959.0


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.4 Plot a color map of average profit per county in 2015.

[Check out this site](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/) for  information on how to create map visualizations in python. There are other resources online for map plotting in python as well.

Another interesting map plot might show the difference in profit between time periods. Change in profit from November to December, for example.

### 5.5  Does profit scale proportionally to store size?

Size of a store can be defined as the total gallons sold in 2015. Is the profit to store size ratio increasing as store size increases? Decreasing? Consistent?

### 5.6 Create a broader category for liquor type.

Liquor types are pretty granular in this dataset. Create a column that categorizes these types into a smaller amount of categories. The categories you decide on are up to you.

### 5.7 Do relationships exist between the broader liquor type and profit?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.8 Are there any noticeable relationships between liquor brand and sales? Profit?

### 5.9 The tax board wants to design a tax or taxes that affect larger stores more than smaller "mom and pop" stores.

Based on your investigations into the data, come up with a way you could design a tax that achieves this goal **without explicitly taxing stores based on size or county critera.** The liqour board does not want to obviously punish larger stores or speific counties for fear of backlash, but is willing to tax hard alcohol more than beer, for example.

Feel free to do more EDA if it helps.

Your report should describe whether such a tax is possible or not, and the specifics of what the tax will target/do.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Market research for new store locations (Scenario 2)

A liquor store owner in Iowa is looking to open a store in a new location and has hired you to investigate the market data for potential new locations. The business owner is interested in what you can discover from the data that will help him evaluate a good location for a new storefront.

---

### 6.1 Create columns for Q1 2015 sales and Q1 2016 sales.

The owner is not just interested in what locations are currently selling well, but also which areas are showing the most promising sales growth from 2015 to 2016. 

For yearly changes we will focus on the changes from Q1 2015 to Q1 2016 specifically (avoiding effects of season changes).

Note: Q1 is months: January, February, and March.


In [10]:
iowa['sale_month'] = iowa.Date.dt.month
monthly_sales = pd.pivot_table(iowa, index=['Date'], values=['Sale_Dollar'],aggfunc=[np.sum])
monthly_sales
iowa_monthly_sales = iowa.groupby(['sale_month', 'sale_year']).sum()
q1_sales = iowa_monthly_sales['Sale_Dollar'].ix[1:3]


### 6.2 Decide on a metric for performance that combines Q1 2016 sales as well as growth in sales from Q1 2015.

Again, the owner wants to know that his store will be in a good position to sell a large amount of liquor now, but also have his store be more likely to grow and sell even more going forward. 

In particular, he is worried about opening a store in an area that is showing a decline in liquor sales between Q1 2015 and Q1 2016.

In [11]:
store_monthly_sales = iowa.groupby(['sale_month', 'sale_year', 'Store Number'])['Sale_Dollar'].sum().ix[1:3]
q1_sales = iowa_monthly_sales['Sale_Dollar'].ix[1:3]
total_monthly_sale = iowa.groupby('sale_month')['Sale_Dollar'].sum()
print q1_sales
print total_monthly_sale

sale_month  sale_year
1           2015         1858000.63
            2016         1880583.53
2           2015         2037903.48
            2016         2214465.01
3           2015         2257891.78
            2016         2304449.02
Name: Sale_Dollar, dtype: float64
sale_month
1     3738584.16
2     4252368.49
3     4562340.80
4     2302566.62
5     2259716.10
6     2754960.95
7     2175922.14
8     2164753.72
9     2387918.92
10    2810088.85
11    2460865.60
12    3056656.60
Name: Sale_Dollar, dtype: float64


### 6.3 Recommend a location based on your metric.

The owner asks you to make a recommendation for a new store location. Explain why you recommended this area.

In [12]:
iowa.groupby('City').sum().sort(['Profit'], ascending=False).head()

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Bottle_Volume_ml,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale_Dollar,Volume_Sold_Liters,Volume_Sold_Gallons,sale_year,Profit,Gallon_profit,sale_month
City,,,,,,,,,,,
DES MOINES,20041981,237261.90,356193.92,345375,4405523.20,276952.36,73176.85,47808530,1470751.39,600245.234344,135631
CEDAR RAPIDS,15960728,178649.80,268239.84,196408,2511466.53,172193.46,45494.50,38062884,839165.98,455278.898693,111350
DAVENPORT,9856075,105846.76,158933.70,154617,1716368.36,117013.63,30917.08,23336065,573073.66,274511.377101,66962
IOWA CITY,6948403,79758.02,119759.75,94095,1254420.29,80854.97,21361.41,16022764,418960.20,193215.426659,45509
SIOUX CITY,7261125,79715.21,119668.33,92061,1220094.37,81678.16,21580.36,16105464,407652.39,188096.842709,46943


Based on the data, I would recommend opening a liquor store in Des Moines. It by far has the most sales in 
volume and profit is excellent as well

### 6.4 Recommend a location based also on a restricted inventory type.

Your employer has now decided he wants his store to focus on one type of liquor only. He askes you:

1. What would be the best type of liquor to sell to maximize profit?
2. Should the recommended location change if he is only selling one type of liquor?

In [13]:
top_liquor = iowa.groupby(['Item Description', 'City']).sum().sort(['Bottles Sold'], ascending=False)
top_liquor

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,,Bottle_Volume_ml,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale_Dollar,Volume_Sold_Liters,Volume_Sold_Gallons,sale_year,Profit,Gallon_profit,sale_month
Item Description,City,,,,,,,,,,,
Black Velvet,DES MOINES,446375,2876.61,4342.15,11035,81949.15,8279.63,2187.69,1021715,27699.40,6814.860488,2860
Fireball Cinnamon Whiskey,DES MOINES,289150,2986.88,4480.91,10672,186333.38,12422.10,3281.77,570303,62128.02,6135.724977,1742
Captain Morgan Spiced Rum,DES MOINES,202125,2257.50,3386.52,10331,175212.59,9971.19,2633.99,417151,58433.84,4678.835910,1238
Juarez Tequila Gold,DES MOINES,60750,448.46,672.69,10084,104946.18,10072.50,2660.85,118893,34982.06,829.089161,342
Hawkeye Vodka,DES MOINES,557725,2377.75,3568.23,9970,57195.27,8815.90,2329.27,1082160,19084.16,4543.709749,3060
Barton Vodka,DES MOINES,235450,956.41,1435.20,9444,57557.79,9579.51,2530.58,415144,19214.20,1653.816060,1100
Seagrams Extra Dry Gin,WATERLOO,77875,655.60,984.32,9329,33752.23,2436.05,643.86,370799,11281.84,3207.057337,1107
Phillips Vodka,DES MOINES,217250,952.89,1429.76,8710,58825.20,9072.25,2396.52,334520,19635.17,1391.955983,1006
Black Velvet,CEDAR RAPIDS,386100,2474.80,3734.54,8459,77272.39,8286.16,2189.22,868554,26366.53,5757.750738,2459


Black Velvet is the best sold liquor by volume and Des Moines is one of the cities that sells the most of it.
To increase sales, this new liquor store in Des Moines should stock heavy on Black Velvet. Hawkeye Vodka is another
that sells well as well.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Time-related effects

---

You could imagine that liquor sales might be affected by a variety of effects related to time. Do people buy more beer in the summer? Do liquor sales skyrocket in december? Do people buy less liquor on Tuesdays?

You have the date of sales in your dataset, which you can use to pull out time components.

1. Come up with 3 different hypotheses about how liquor sales may vary with time-related variables. 
2. Create a visualization exploring each hypothesis.
3. Write brief concluding remarks on what you observed.


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Appropriate spending

---

The owner you worked with before realized that opening new stores cost money, something he had completely overlooked earlier. He brings you back in to help him figure out how much he should be willing to spend on opening his store.

### 8.1 Calculate costs and time to recoup.

The owner has decided he will buy 20,000 gallons of the liquor type you recommended (randomly sampled across the different brands/subtypes). He will also be opening his store in the area you recommended.

The cost of buying the property for his store will be $150,000.

1. How much should he expect to spend on the 20,000 gallons of liquor alone?
2. Based on the amount he's spent on the liquor and property, and on the profit per time period he is expected to have, how long will it take him to recoup the costs?

<img src="http://imgur.com/gGrau8s.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">


### 8.2 Calculate maximum loan interest rate.

The owner unfortunately has to take out a loan to to pay for the entirety of the property and liquor. 

**Conditions of the loan**:

Every quarter, (except for Q1 when he takes out the loan), the interest rate is applied to the remaining balance on the loan and added to his debt. 

If, for example, he made no payments whatsoever, not only would the amount of debt he owed keep increasing by quarter but the debt incurred from interest would keep increasing as well. (Since his debt keeps growing, the interest rate is applied each month to the bigger balance, incurring more debt each time.)

**Determining maximum acceptable interest:**

Given this, _the owner wants to be 99% confident that he will be able to pay off his loans._ If the interest rate is too high he will have to eventually declare bankruptcy.

Based on his expected profits per quarter (assume the expected profit stays the same for all future quarters), what is the maximum interest rate he should accept on the loan to be 99% confident he will be able to pay it off?